**AVISO**: Notebook en elaboración e revisión. Poden conter fallos.

# Conexión a MS-SQL
## Empregando pyodbc

### Que é ODBC
As siglas significan Open Database Connectivity. É un estándar para acceder ás bases de datos.

Seguramente coñezas tamén: JDBC (Java Database Connectivity) que está baseado na mesma idea do estándar anterior.

Máis información: <https://es.wikipedia.org/wiki/Open_Database_Connectivity>

### Instalando MS-SQL
Para instalar Microsoft SQL Server, empregaremos o docker dos apuntes

### Instalación da libraría pyodbc

Dende o contorno bigdata de conda, instalamos pyodbc de conda-forge:


## Instalación Driver

### **Microsoft Windows**

Instalar el Driver desde la página del enlace con la versión **18**

<a href="https://learn.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver16#download-for-windows">ODBC Driver .msi para Windows</a>

### **GNU/Linux Debian**

#### 1. Instalar clave pública y confiar en ella:

  ~~~
  curl https://packages.microsoft.com/keys/microsoft.asc | sudo tee /etc/apt/trusted.gpg.d/microsoft.asc
  ~~~

#### 2. Elegir versión:

  - Debian 9:
  ~~~~
  curl https://packages.microsoft.com/config/debian/9/prod.list | sudo tee /etc/apt/sources.list.d/mssql-release.list
  ~~~~

  - Debian 10:

  ~~~~
  curl https://packages.microsoft.com/config/debian/10/prod.list | sudo tee /etc/apt/sources.list.d/mssql-release.list
  ~~~~

  - Debian 11:

  ~~~~
  curl https://packages.microsoft.com/config/debian/11/prod.list | sudo tee /etc/apt/sources.list.d/mssql-release.list
  ~~~~

  - Debian 12:

  ~~~~
  curl https://packages.microsoft.com/config/debian/12/prod.list | sudo tee /etc/apt/sources.list.d/mssql-release.list
  ~~~~

Para todos:

  ~~~~
  sudo apt-get update
  sudo ACCEPT_EULA=Y apt-get install -y msodbcsql18
  ~~~~

- Opcional 1: bcp y sqlcmd

~~~~
sudo ACCEPT_EULA=Y apt-get install -y mssql-tools18
echo 'export PATH="$PATH:/opt/mssql-tools18/bin"' >> ~/.bashrc
source ~/.bashrc
~~~~

- Opcional 2: Cabeceras de unixODBC para desarrollo

~~~~
sudo apt-get install -y unixodbc-dev
~~~~

- Opcional 3: Librería de Kerberos para las distros debian-slim

~~~~
sudo apt-get install -y libgssapi-krb5-2
~~~~

In [1]:
!conda install -c conda-forge -y pyodbc

Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.



#### Importamos a libraría

In [ ]:
import pyodbc

#### Datos de conexión

In [ ]:
DB_HOST = '127.0.0.1,41433'
DB_NAME = 'employees'
DB_USER = 'employee1'
DB_PASSWORD = 'strong_password'

#### Establecemos a conexión

Primeiro creamos a cadea de conexión e logo chamamos ao método connect.

In [ ]:
connectionString = f'''DRIVER={{ODBC Driver 18 for SQL Server}};
SERVER={DB_HOST};DATABASE={DB_NAME};
UID={DB_USER};PWD={DB_PASSWORD};
Encrypt=Yes;TrustServerCertificate=Yes'''

conexion = pyodbc.connect(connectionString)

#### Executando unha query de lectura

In [ ]:
SQL_QUERY = """
SELECT 
nome, apelidos, dni
FROM
empregado
"""

#Creamos un cursos para logo acceder aos resultados
cursor = conexion.cursor()

#Finalmente executamos a consulta
cursor.execute(SQL_QUERY)

tuplas = cursor.fetchall()
for tupla in tuplas:
    print(f"{tupla.apelidos}, {tupla.nome} \t - \t {tupla.dni}")

#### Insertando unha fila

O seguinte modo de consulta, facemos un un prepare statement ou consulta parametrizada para previr o SQL Injection. Os ? substituímolos por variables e executamos esa consulta.

In [ ]:
SQL_PREPARED_STATEMENT = """
INSERT INTO empregado(nome, apelidos, dni, data_creacion)
VALUES (?,?,?, CURRENT_TIMESTAMP)
"""
sufixo="1"

cursor.execute(
    SQL_PREPARED_STATEMENT,
    f'nome_{sufixo}', 
    f'apelidos_{sufixo}', 
    '12345678A')

# Alternativa
valores = (f'nome_{sufixo}', 
           f'apelidos_{sufixo}', 
           '12345678A')
cursor.execute(SQL_PREPARED_STATEMENT, valores)

# Obtemos o id de resultado, para ver se todo está correcto
id_resultado = cursor.fetchval()
print(id_resultado)

# Debemos confirmar cun commit a transacción:
conexion.commit()
# conexion.rollback()

#Pechar o cursor e a conexión
cursor.close()
conexion.close()

#### Xuntalo con PANDAS é moi simple

In [ ]:
import pandas as pd

query_for_pandas="SELECT * FROM empregado"
conexion_p = pyodbc.connect(connectionString)
df = pd.read_sql_query(query_for_pandas, conexion_p)

conexion_p.close()

df

Máis información:
  - <https://pieriantraining.com/python-tutorial-how-to-connect-to-sql-server-in-python/>
  - <https://learn.microsoft.com/es-es/sql/connect/python/pyodbc/step-3-proof-of-concept-connecting-to-sql-using-pyodbc?view=sql-server-ver16>
  - <https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver16&tabs=debian18-install%2Calpine17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline>